In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from spieks.network.converter import swap_layers
from spieks.ann.neurons import QCFS
from spieks.ann.training import test_ann, train_ann

print(f"Using pytorch {torch.__version__}")

N = 512
T = 5
DT = 0.1

MAX_HZ = 10

BS = 32
LR = 1e-4
EPOCHS = 40

# Check if CUDA is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device ({DEVICE})')

Using pytorch 2.5.1
Using device (cuda)


Seeding

In [2]:
SEED = 42

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

Define the model

In [3]:
class MNISTModel(nn.Module):
	def __init__(self):
		super(MNISTModel, self).__init__()
		self.flatten = nn.Flatten()
		self.w1 = nn.Linear(28 * 28, 128)
		self.r1 = nn.ReLU()
		self.w2 = nn.Linear(128, 64)
		self.r2 = nn.ReLU()
		self.drop = nn.Dropout(0.5)
		self.w3 = nn.Linear(64, 10)

		nn.init.kaiming_uniform_(self.w1.weight, nonlinearity='relu')
		nn.init.kaiming_uniform_(self.w2.weight, nonlinearity='relu')
		nn.init.kaiming_uniform_(self.w3.weight, nonlinearity='relu')

	def forward(self, x):
		x = self.flatten(x)
		x = self.r1(self.w1(x))
		x = self.r2(self.w2(x))
		x = self.drop(x)
		x = self.w3(x)
		return x

Train the base ANN on MNSIT

In [4]:
model_path = "tmp/models"
model_file = f"{model_path}/mnist.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

train_dataset = torchvision.datasets.MNIST(root='./tmp/data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./tmp/data/', train=False, transform=transform, download=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, drop_last=True, pin_memory=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=True, drop_last=True, pin_memory=True, num_workers=8)

# Check if CUDA is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device ({DEVICE})')

# Initialize model
model = MNISTModel().to(DEVICE)

# Load or train model
if os.path.exists(model_file):
	# Load model if it exists
	model.load_state_dict(torch.load(model_file, weights_only=True))
	print(f"Model loaded from file: {model_file}")
else:
	# Train from scratch otherwise
	loss_fn = nn.CrossEntropyLoss()

	# Replace all ReLU layers with QCFS to improve SNN inference
	swap_layers(model, old_layer_type=nn.ReLU, new_layer_type=QCFS, neuron_args={ "Q": 8 })
	
	model = train_ann(model, train_loader, test_loader, loss_fn, EPOCHS, DEVICE, save_path=model_file)

# Calculate the ANN loss and accuracy
ann_loss, ann_acc = test_ann(model, DEVICE, test_loader, loss_fn=nn.CrossEntropyLoss())
print(f"ANN Loss (CrossEntropy): {ann_loss}")
print(f"ANN Accuracy (CrossEntropy): {ann_acc}")

Using device (cuda)


  0%|          | 0/40 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'tmp/models/mnist.txt'

Convert the ANN to an SNN

In [ ]:
from spieks.network.converter import Converter
from spieks.neurons import IF
print(model)
spiking_model = Converter.convert(model, DT, model_subs={ nn.ReLU: IF })
print(spiking_model)

Evaluate the SNN on classification

In [ ]:
from spieks.simulator import Classifier
classifier = Classifier(spiking_model, device=DEVICE)
activations, loss, accuracy = classifier.evaluate_dataset(test_loader, duration=T)
final_accuracy = accuracy[-1]
print(f"Accuracy: {final_accuracy}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

TIME = np.arange(len(accuracy)) * DT

plt.figure()
plt.title("Accuracy")
plt.hlines(ann_acc, 0, T, linestyles='--', color='C1', label='ReLU Baseline')
plt.plot(TIME, accuracy, label='IF')
plt.figure()

plt.figure()
plt.title("Loss")
plt.plot(TIME, np.log(loss), label='IF')
plt.figure()
plt.show()